In [10]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

In [11]:
# Function for comparing different models
def score_model(model, x_train, x_valid, y_train, y_valid):
    model.fit(x_train, y_train)
    preds = model.predict(x_valid)
    return mean_absolute_error(y_valid, preds)

In [12]:
# read data
test_data = pd.read_csv("test data.csv")
train_data = pd.read_csv("train data.csv")

In [13]:
# train data
# use only numerical data
x = train_data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])
# Create target object
y = train_data.SalePrice

In [14]:
# Divide data into training and validation data
x_train, x_valid, y_train, y_valid = train_test_split(x, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [15]:
# check how many missing values by column
missing_values = (x_train.isnull().sum())
print(missing_values[missing_values > 0])

LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [16]:
# Missing values: Drop columns
# get names of columns with missing values
cols_with_missing = [col for col in x_train.columns
                     if x_train[col].isnull().any()]
# Drop columns in training and validation data
reduced_x_train = x_train.drop(cols_with_missing, axis=1)
reduced_x_valid = x_valid.drop(cols_with_missing, axis=1)

# get mae
print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_x_train, reduced_x_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):
19003.3198630137


In [17]:
# Missing values: Imputation
my_imputer = SimpleImputer()
imputed_x_train = pd.DataFrame(my_imputer.fit_transform(x_train))
imputed_x_valid = pd.DataFrame(my_imputer.transform(x_valid))

# Imputation removed column names; put them back
imputed_x_train.columns = x_train.columns
imputed_x_valid.columns = x_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_x_train, imputed_x_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
19243.57294520548


In [18]:
# Missing values: extended imputation
# Make copy to avoid changing original data (when imputing)
x_train_plus = x_train.copy()
x_valid_plus = x_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    x_train_plus[col + ' was missing'] = x_train_plus[col].isnull()
    x_valid_plus[col + ' was missing'] = x_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_x_train_plus = pd.DataFrame(my_imputer.fit_transform(x_train_plus))
imputed_x_valid_plus = pd.DataFrame(my_imputer.transform(x_valid_plus))

# Imputation removed column names; put them back
imputed_x_train_plus.columns = x_train_plus.columns
imputed_x_valid_plus.columns = x_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_x_train_plus, imputed_x_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
19349.617465753425
